In [1]:
%load_ext autoreload
%autoreload 2

# The Dataset: Basecamp Handbook

(give the resources and intro here)

In [2]:
!tree data/

data/
├── 37signals-is-you.md
├── benefits-and-perks.md
├── code-of-conduct.md
├── faq.md
├── getting-started.md
├── how-we-work.md
├── international-travel-guide.md
├── LICENSE.md
├── making-a-career.md
├── managing-work-devices.md
├── moonlighting.md
├── our-internal-systems.md
├── our-rituals.md
├── performance-plans.md
├── product-histories.md
├── README.md
├── stateFMLA.md
├── titles-for-data.md
├── titles-for-designers.md
├── titles-for-ops.md
├── titles-for-programmers.md
├── titles-for-support.md
├── vocabulary.md
├── what-influenced-us.md
├── what-we-stand-for.md
└── where-we-work.md

0 directories, 26 files


## Generate Synthetic Testset

1. load the data as documents
2. generate the testset
3. upload and verify it with langsmith
4. good to go

In [6]:
from langchain.document_loaders import DirectoryLoader
loader = DirectoryLoader("./data/")
documents = loader.load()

# add filename as metadata
for document in documents:
    document.metadata['file_name'] = document.metadata['source']

docs = documents
# how many docs do we have
len(docs)

26

In [7]:
# create the TestsetGenerator
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context

# generator with openai models
generator = TestsetGenerator.with_openai()

# generate testset
testset = generator.generate_with_langchain_docs(
    documents, 
    test_size=6, 
    distributions={simple: 0.5, reasoning: 0.25, multi_context: 0.25}
)

embedding nodes:   0%|          | 0/70 [00:00<?, ?it/s]

Filename and doc_id are the same for all nodes.


Generating:   0%|          | 0/7 [00:00<?, ?it/s]

In [8]:
test_df = testset.to_pandas()
test_df.head()

,question,contexts,ground_truth,evolution_type,episode_done
0,What is the value of being straightforward in ...,[What We Stand For\n\nValues\n\nBefore anythin...,Being straightforward in the company's approac...,simple,True
1,How does 37signals determine pay and promotions?,[Making A Career\n\nYour First 90 Days\n\nCong...,37signals determines pay based on Radford mark...,simple,True
2,What is the significance of limiting the work ...,[als better.\n\nAs soon as organizational bott...,Limiting the work week to 40 hours helps prior...,simple,True
3,How does a presentable scope of work contribut...,[How We Work\n\nCycles\n\nWe work in 6-week or...,A presentable scope of work contributes to eff...,reasoning,True
4,What are the responsibilities and qualificatio...,[Titles for Data Analysts\n\nWe use the follow...,The responsibilities of a Lead Data Analyst at...,reasoning,True


In [ ]:
from ragas.integrations.langsmith import upload_dataset

dataset_name = "basecamp"
dataset_desc = "Synthetic testset data for basecamp"

upload_dataset(testset, dataset_name, dataset_desc)

In [ ]:
from langsmith import Client
from langsmith.utils import LangSmithNotFoundError

client = Client()

In [ ]:
# upload to langsmith

try:
    # check if dataset exists
    dataset = client.read_dataset(dataset_name=dataset_name)
    print("using existing dataset: ", dataset.name)
except LangSmithNotFoundError:
    # if not create a new one with the generated query examples    
    dataset = client.upload_dataframe(
        df=test_df, 
        name=dataset_name, 
        input_keys=["question"], 
        output_keys=["ground_truth"],
        description=dataset_desc
    )
    
    print("Created a new dataset: ", dataset.name)

In [ ]:
from ragas.integrations.langsmith import upload_dataset